In [16]:
from transformers import pipeline
from pprint import pprint

In [2]:
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

model.safetensors: 100%|██████████| 499M/499M [00:09<00:00, 54.3MB/s] 
e:\JoshuaLe\Github\rest-api-demo\venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JOSHUALE\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
tokenizer_config.json: 100%|██████████| 380/380 [00:00<?, ?B/s

In [3]:
classifier.save_pretrained('./models/pretrained/roberta-base-go_emotions')

In [4]:
loaded_classifier = pipeline(task="text-classification", model="./models/pretrained/roberta-base-go_emotions", top_k=None)

In [8]:
results = loaded_classifier.predict(['I love football', 'I hate hot weather'])

In [10]:
summarizer = pipeline("summarization", model="Falconsai/text_summarization")
summarizer.save_pretrained('./models/pretrained/t5_small_text_summarization')

config.json: 100%|██████████| 1.49k/1.49k [00:00<00:00, 1.49MB/s]
model.safetensors: 100%|██████████| 242M/242M [00:12<00:00, 20.1MB/s] 
generation_config.json: 100%|██████████| 112/112 [00:00<?, ?B/s] 
tokenizer_config.json: 100%|██████████| 2.32k/2.32k [00:00<?, ?B/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 6.00MB/s]
tokenizer.json: 100%|██████████| 2.42M/2.42M [00:01<00:00, 1.64MB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<?, ?B/s]


In [11]:
loaded_summarizer = pipeline("summarization", model='./models/pretrained/t5_small_text_summarization')

In [12]:
ARTICLE = """ 
Hugging Face: Revolutionizing Natural Language Processing
Introduction
In the rapidly evolving field of Natural Language Processing (NLP), Hugging Face has emerged as a prominent and innovative force. This article will explore the story and significance of Hugging Face, a company that has made remarkable contributions to NLP and AI as a whole. From its inception to its role in democratizing AI, Hugging Face has left an indelible mark on the industry.
The Birth of Hugging Face
Hugging Face was founded in 2016 by Clément Delangue, Julien Chaumond, and Thomas Wolf. The name "Hugging Face" was chosen to reflect the company's mission of making AI models more accessible and friendly to humans, much like a comforting hug. Initially, they began as a chatbot company but later shifted their focus to NLP, driven by their belief in the transformative potential of this technology.
Transformative Innovations
Hugging Face is best known for its open-source contributions, particularly the "Transformers" library. This library has become the de facto standard for NLP and enables researchers, developers, and organizations to easily access and utilize state-of-the-art pre-trained language models, such as BERT, GPT-3, and more. These models have countless applications, from chatbots and virtual assistants to language translation and sentiment analysis.
Key Contributions:
1. **Transformers Library:** The Transformers library provides a unified interface for more than 50 pre-trained models, simplifying the development of NLP applications. It allows users to fine-tune these models for specific tasks, making it accessible to a wider audience.
2. **Model Hub:** Hugging Face's Model Hub is a treasure trove of pre-trained models, making it simple for anyone to access, experiment with, and fine-tune models. Researchers and developers around the world can collaborate and share their models through this platform.
3. **Hugging Face Transformers Community:** Hugging Face has fostered a vibrant online community where developers, researchers, and AI enthusiasts can share their knowledge, code, and insights. This collaborative spirit has accelerated the growth of NLP.
Democratizing AI
Hugging Face's most significant impact has been the democratization of AI and NLP. Their commitment to open-source development has made powerful AI models accessible to individuals, startups, and established organizations. This approach contrasts with the traditional proprietary AI model market, which often limits access to those with substantial resources.
By providing open-source models and tools, Hugging Face has empowered a diverse array of users to innovate and create their own NLP applications. This shift has fostered inclusivity, allowing a broader range of voices to contribute to AI research and development.
Industry Adoption
The success and impact of Hugging Face are evident in its widespread adoption. Numerous companies and institutions, from startups to tech giants, leverage Hugging Face's technology for their AI applications. This includes industries as varied as healthcare, finance, and entertainment, showcasing the versatility of NLP and Hugging Face's contributions.
Future Directions
Hugging Face's journey is far from over. As of my last knowledge update in September 2021, the company was actively pursuing research into ethical AI, bias reduction in models, and more. Given their track record of innovation and commitment to the AI community, it is likely that they will continue to lead in ethical AI development and promote responsible use of NLP technologies.
Conclusion
Hugging Face's story is one of transformation, collaboration, and empowerment. Their open-source contributions have reshaped the NLP landscape and democratized access to AI. As they continue to push the boundaries of AI research, we can expect Hugging Face to remain at the forefront of innovation, contributing to a more inclusive and ethical AI future. Their journey reminds us that the power of open-source collaboration can lead to groundbreaking advancements in technology and bring AI within the reach of many.
"""

summary = summarizer(ARTICLE, max_length=230, min_length=30, do_sample=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors


In [17]:
pprint(summary)

[{'summary_text': 'Hugging Face has emerged as a prominent and innovative '
                  'force in NLP . From its inception to its role in '
                  'democratizing AI, the company has left an indelible mark on '
                  'the industry . The name "Hugging Face" was chosen to '
                  "reflect the company's mission of making AI models more "
                  'accessible and friendly to humans .'}]
